In [20]:
import tensorflow as tf
import pandas as pd
from pathlib import Path

print("Visible devices:", tf.config.list_physical_devices())

Visible devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [21]:
def make_dataset(split_dir, img_size=(224, 224), batch_size=8, shuffle=True):
    split_dir = Path(split_dir)
    csv_path = split_dir / "_classes.csv"

    df = pd.read_csv(csv_path)
    filepaths = df["filename"].apply(lambda fname: str(split_dir / fname)).values
    labels = df.drop(columns=["filename"]).values.astype("float32")
    num_classes = labels.shape[1]

    ds = tf.data.Dataset.from_tensor_slices((filepaths, labels))

    if shuffle:
        ds = ds.shuffle(buffer_size=min(512, len(filepaths)))  # 512 is plenty

    def load_image(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, img_size)        # 👈 resize here!
        img = tf.cast(img, tf.float32) / 255.0
        return img, label

    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return ds, num_classes

In [22]:
train_ds, num_classes = make_dataset("dataset/train", batch_size=8, shuffle=True)
valid_ds, _          = make_dataset("dataset/valid", batch_size=8, shuffle=False)
test_ds, _           = make_dataset("dataset/test",  batch_size=8, shuffle=False)

In [23]:
for batch_images, batch_labels in train_ds.take(1):
    print("Batch images:", batch_images.shape)
    print("Batch labels:", batch_labels.shape)

Batch images: (8, 224, 224, 3)
Batch labels: (8, 25)


2025-11-15 00:11:02.383767: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [24]:
IMG_SIZE = (224, 224)
INPUT_SHAPE = IMG_SIZE + (3,)

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=INPUT_SHAPE,
    pooling="avg",  # gives a 2048-d feature vector
)

# First: freeze the base model (only train new head)
base_model.trainable = False

inputs = tf.keras.Input(shape=INPUT_SHAPE)
# x = tf.keras.applications.efficientnet.preprocess_input(inputs)
x = inputs

x = base_model(x, training=False)

x = tf.keras.layers.Dense(256, activation="relu")(x)
# x = tf.keras.layers.Dense(512, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

# Multi-label output → sigmoid
outputs = tf.keras.layers.Dense(num_classes, activation="sigmoid")(x)

model = tf.keras.Model(inputs, outputs)


In [25]:
class HammingLoss(tf.keras.metrics.Metric):
    def __init__(self, name="hamming_loss", threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.total_loss = self.add_weight(name="total_loss", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert probabilities → 0/1 labels
        y_pred_bin = tf.cast(y_pred >= self.threshold, tf.float32)

        # Compare predictions vs labels
        mismatches = tf.not_equal(y_true, y_pred_bin)
        mismatches = tf.cast(mismatches, tf.float32)

        # Hamming loss per sample: mean mismatches across classes
        sample_loss = tf.reduce_mean(mismatches, axis=1)

        # Accumulate
        self.total_loss.assign_add(tf.reduce_sum(sample_loss))
        self.count.assign_add(tf.cast(tf.size(sample_loss), tf.float32))

    def result(self):
        return self.total_loss / self.count

    def reset_state(self):
        self.total_loss.assign(0.0)
        self.count.assign(0.0)

class MultiLabelAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name="multilabel_accuracy", threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.total_acc = self.add_weight(name="total_acc", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Turn predictions into binary decisions
        y_pred_bin = tf.cast(y_pred >= self.threshold, tf.float32)

        # Label-wise correctness: shape (batch, num_labels)
        matches = tf.cast(tf.equal(y_true, y_pred_bin), tf.float32)

        # Per-sample accuracy: shape (batch,)
        sample_acc = tf.reduce_mean(matches, axis=1)

        # Accumulate
        self.total_acc.assign_add(tf.reduce_sum(sample_acc))
        self.count.assign_add(tf.cast(tf.shape(sample_acc)[0], tf.float32))

    def result(self):
        return self.total_acc / self.count

    def reset_state(self):
        self.total_acc.assign(0.0)
        self.count.assign(0.0)



In [26]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=[
        HammingLoss(),
        tf.keras.metrics.AUC(name="auc"),
        MultiLabelAccuracy(),
    ],
)

In [27]:
history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=5,        # start small, just to get the head in a good place
)

Epoch 1/5
286/286 ━━━━━━━━━━━━━━━━━━━━ 19s 52ms/step - auc: 0.7313 - hamming_loss: 0.3051 - loss: 0.5549 - multilabel_accuracy: 0.6949 - val_auc: 0.7803 - val_hamming_loss: 0.2663 - val_loss: 0.5039 - val_multilabel_accuracy: 0.7337
Epoch 2/5
286/286 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - auc: 0.7578 - hamming_loss: 0.2894 - loss: 0.5280 - multilabel_accuracy: 0.7106 - val_auc: 0.7780 - val_hamming_loss: 0.2717 - val_loss: 0.5024 - val_multilabel_accuracy: 0.7283
Epoch 3/5
286/286 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - auc: 0.7628 - hamming_loss: 0.2864 - loss: 0.5220 - multilabel_accuracy: 0.7136 - val_auc: 0.7778 - val_hamming_loss: 0.2663 - val_loss: 0.5013 - val_multilabel_accuracy: 0.7337
Epoch 4/5
286/286 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - auc: 0.7651 - hamming_loss: 0.2847 - loss: 0.5189 - multilabel_accuracy: 0.7153 - val_auc: 0.7785 - val_hamming_loss: 0.2663 - val_loss: 0.5010 - val_multilabel_accuracy: 0.7337
Epoch 5/5
286/286 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - auc: 0.7653 -

In [28]:
# Unfreeze the base model
base_model.trainable = True

# Option 1: freeze most layers, only train last ~30
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Use a lower LR for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=[
        HammingLoss(),
        tf.keras.metrics.AUC(name="auc"),
        MultiLabelAccuracy(),
    ],
)

history_ft = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=5,   # or more if it’s stable
)

Epoch 1/5
286/286 ━━━━━━━━━━━━━━━━━━━━ 38s 111ms/step - auc: 0.7543 - hamming_loss: 0.2898 - loss: 0.5274 - multilabel_accuracy: 0.7102 - val_auc: 0.7783 - val_hamming_loss: 0.2663 - val_loss: 0.5020 - val_multilabel_accuracy: 0.7337
Epoch 2/5
286/286 ━━━━━━━━━━━━━━━━━━━━ 27s 95ms/step - auc: 0.7643 - hamming_loss: 0.2842 - loss: 0.5205 - multilabel_accuracy: 0.7158 - val_auc: 0.7805 - val_hamming_loss: 0.2663 - val_loss: 0.4997 - val_multilabel_accuracy: 0.7337
Epoch 3/5
286/286 ━━━━━━━━━━━━━━━━━━━━ 27s 93ms/step - auc: 0.7638 - hamming_loss: 0.2843 - loss: 0.5203 - multilabel_accuracy: 0.7157 - val_auc: 0.7770 - val_hamming_loss: 0.2663 - val_loss: 0.5009 - val_multilabel_accuracy: 0.7337
Epoch 4/5
286/286 ━━━━━━━━━━━━━━━━━━━━ 28s 96ms/step - auc: 0.7629 - hamming_loss: 0.2873 - loss: 0.5207 - multilabel_accuracy: 0.7127 - val_auc: 0.7809 - val_hamming_loss: 0.2657 - val_loss: 0.4996 - val_multilabel_accuracy: 0.7343
Epoch 5/5
286/286 ━━━━━━━━━━━━━━━━━━━━ 28s 99ms/step - auc: 0.7608 